In [19]:
import numpy as np 
import torchvision.datasets as datasets

train_set = datasets.MNIST('./data', train=True, download=True)
test_set = datasets.MNIST('./data', train=False, download=True)

In [23]:
train = []
train_set_array = train_set.data.numpy() 

for i in range(len(train_set_array)): 
    train.append((train_set_array[i].reshape(-1, 1), train_set.targets.numpy()[i]))

np.random.shuffle(train) 

N = len(train) 
X_train = np.hstack(tuple([train[i][0] for i in range(40000)])) / 255.
Y_train = np.hstack(tuple([train[i][1] for i in range(40000)]))
X_test = np.hstack(tuple([train[i][0] for i in range(40000, N)])) / 255.
Y_test = np.hstack(tuple([train[i][1] for i in range(40000, N)]))


In [24]:
def initializer(): 
    W1 = np.random.uniform(-1, 1, size=(10, 784)) 
    b1 = np.random.uniform(-1, 1, size=(10, 1)) 
    W2 = np.random.uniform(-1, 1, size=(10, 10)) 
    b2 = np.random.uniform(-1, 1, size=(10, 1)) 
    
    return W1, b1, W2, b2 

def ReLU(Z): 
    return np.maximum(0, Z)

def ReLU_d(Z): 
    return Z > 0

def softMax(X:np.array): 
    x_max = np.max(X, axis=0)
    X = X - x_max
    return np.exp(X) / np.sum(np.exp(X), axis=0) 

def softMax_d(X:np.array): 
    return np.diag(X) - np.matmul(X, np.transpose(X))


def one_hot(Y): 
    one_hot_Y = np.zeros((Y.size, 10))
    one_hot_Y[np.arange(Y.size), Y] = 1 
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def forwardProp(W1, b1, W2, b2, X): 
    Z1 = np.matmul(W1, X) + b1
    A1 = ReLU(Z1) 
    Z2 = np.matmul(W2, A1) + b2 
    A2 = softMax(Z2) 
    return Z1, A1, Z2, A2 

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    n, m = X.shape
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, 1)
    dZ1 = W2.T.dot(dZ2) * ReLU_d(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, 1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1.reshape(10, 1)    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2.reshape(10, 1)    
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = initializer()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forwardProp(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [26]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[1 5 1 ... 1 5 1] [5 9 5 ... 6 8 0]
0.132025
Iteration:  10
[5 5 3 ... 3 6 0] [5 9 5 ... 6 8 0]
0.304725
Iteration:  20
[5 4 3 ... 3 6 0] [5 9 5 ... 6 8 0]
0.3733
Iteration:  30
[5 4 3 ... 3 6 0] [5 9 5 ... 6 8 0]
0.412325
Iteration:  40
[5 4 3 ... 3 6 0] [5 9 5 ... 6 8 0]
0.438275
Iteration:  50
[5 4 3 ... 3 8 0] [5 9 5 ... 6 8 0]
0.46055
Iteration:  60
[5 4 3 ... 3 8 0] [5 9 5 ... 6 8 0]
0.48
Iteration:  70
[5 4 3 ... 3 8 0] [5 9 5 ... 6 8 0]
0.495925
Iteration:  80
[5 4 3 ... 3 8 0] [5 9 5 ... 6 8 0]
0.5115
Iteration:  90
[5 4 3 ... 3 8 0] [5 9 5 ... 6 8 0]
0.5252
Iteration:  100
[5 4 3 ... 3 8 0] [5 9 5 ... 6 8 0]
0.538825
Iteration:  110
[5 4 3 ... 3 5 0] [5 9 5 ... 6 8 0]
0.55095
Iteration:  120
[5 4 3 ... 3 5 0] [5 9 5 ... 6 8 0]
0.563625
Iteration:  130
[5 9 3 ... 3 5 0] [5 9 5 ... 6 8 0]
0.574125
Iteration:  140
[5 9 3 ... 3 5 0] [5 9 5 ... 6 8 0]
0.5836
Iteration:  150
[9 9 3 ... 3 5 0] [5 9 5 ... 6 8 0]
0.5929
Iteration:  160
[5 9 3 ... 3 5 0] [5 9 5 ... 6 8 0]